# ECON-420 Assignment 2 - Ryan George & Sophia Beronilla

# Import Libraries

In [9]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Load Dataset

In [10]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Normalize pixel values from 0 to 1

In [11]:
train_images, test_images = train_images / 255.0, test_images / 255.0

# Reshape to Greyscale

In [12]:
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

# Train-Test Split

In [13]:
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)

# Upscale for Pre-Trained Model

In [14]:
train_images_upsampled = tf.image.resize(train_images, (32, 32))
val_images_upsampled = tf.image.resize(val_images, (32, 32))
test_images_upsampled = tf.image.resize(test_images, (32, 32))

# Build the training pipeline

In [15]:
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_images_upsampled, train_labels, batch_size=batch_size)
val_generator = val_datagen.flow(val_images_upsampled, val_labels, batch_size=batch_size)
test_generator = test_datagen.flow(test_images_upsampled, test_labels, batch_size=batch_size)


# Model

In [16]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

model = models.Sequential()
model.add(layers.UpSampling2D(size=(2, 2), input_shape=(28, 28, 1)))
model.add(layers.Conv2D(3, (3, 3), activation='relu', padding='same'))
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
history = model.fit(train_generator, epochs=5, validation_data=val_generator)

Epoch 1/5
1500/1500 [==============================] - 161s 106ms/step - loss: 0.2858 - accuracy: 0.9294 - val_loss: 23.1482 - val_accuracy: 0.1102
Epoch 2/5
1500/1500 [==============================] - 162s 108ms/step - loss: 0.1178 - accuracy: 0.9725 - val_loss: 20.3845 - val_accuracy: 0.1102
Epoch 3/5
1500/1500 [==============================] - 145s 96ms/step - loss: 0.0929 - accuracy: 0.9781 - val_loss: 6.0483 - val_accuracy: 0.1079
Epoch 4/5
1500/1500 [==============================] - 147s 98ms/step - loss: 0.0727 - accuracy: 0.9819 - val_loss: 0.4288 - val_accuracy: 0.9377
Epoch 5/5
1500/1500 [==============================] - 155s 103ms/step - loss: 0.0797 - accuracy: 0.9805 - val_loss: 21.7651 - val_accuracy: 0.1460


# Model Evaluation

In [18]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

313/313 [==============================] - 7s 21ms/step - loss: 21.6488 - accuracy: 0.1508
Test accuracy: 0.15080000460147858
